In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from tqdm.notebook import tqdm
from perlin_noise import PerlinNoise
import os

In [ ]:
a = np.random.rand(1000000)
a.shape

In [ ]:
%%time
b = np.sort(a)

In [ ]:
%%time
c = np.sort(b, kind='mergesort')

In [ ]:
size = (100, 200)

# initialize heigh map with perlin noise
height_map0 = np.zeros(size)
x = np.arange(size[0])
y = np.arange(size[1])
Y, X = np.meshgrid(y, x)

noise = PerlinNoise(octaves=3, seed=1)
y_ridge_mean = 20
ridge_wobble = 10
wobble_noise = PerlinNoise(octaves=3, seed=2)
y_ridge = y_ridge_mean + ridge_wobble*np.array([wobble_noise(i/size[0]) for i in range(size[0])])
y_sigma = 10
ridge_height = 1

for i in tqdm(range(size[0])):
    for j in range(size[1]):
        height_map0[i,j] = ridge_height/(1 + (j - y_ridge[i])**2 / y_sigma**2) + noise([i/size[0], j/size[1]]) + 1

# height_map0 = -np.exp(-np.sqrt((X - 0.5*size[0])**2 + (Y-0.5*size[1])**2)/20)


plt.close(1)
fig, ax = plt.subplots(num=1, figsize=(9,6))

plt.imshow(height_map0, cmap='gray')
plt.colorbar()

fig.tight_layout()

In [ ]:
# compute the gradient of the height map
grad = np.gradient(-height_map0)
# height_map0_grad = np.sqrt(height_map0_grad[0]**2 + height_map0_grad[1]**2)


In [ ]:
plt.close(1)
fig, ax = plt.subplots(num=1, figsize=(9,6))

# plt.imshow(height_map0, cmap='gray')
plt.pcolormesh(X,Y,height_map0, cmap='gray')
plt.axis('equal')
plt.colorbar()

plt.quiver(X,Y,grad[0],grad[1], color='red')

fig.tight_layout()

## Erosion dynamics

At each grid-cell $k=(i,j)$ we want to track several quantities:

- The height of the cell $H$
- The water content $Q \geq 0$
- The suspended sediment content $S < \alpha Q$
- The flow velocity $V^x, V^y$

Now we want to write differential equations for all quantities:

$$ \dot{H}_k(t) = deposition - erosion $$

$$ \dot{Q}_k(t) = \Delta_k(t) + flow $$

$$ \dot{S}_k(t) = erosion - deposition + flow $$

$$ \dot{V}_k(t) = height gradient - friction + exported momentum $$

Let us start by simply letting water flow without altering the height map.
Let us also suppose for now that there is no inertia, so that the velocity field is simply the gradient

In [ ]:
Q = np.ones_like(height_map0)
eta = 10


H = np.copy(height_map0)
# Q = np.zeros_like(height_map0)
V = np.stack(np.gradient(-H), axis=-1)

Qs = [np.copy(Q)]
# iterate
for i in tqdm(range(100)):
    # V = np.stack(np.gradient(-H), axis=-1)
    V_mod = np.abs(V[...,0]) + np.abs(V[...,1])
    repartition = np.abs(V[...,0])/V_mod # fraction of flow in x direction at each point
    repartition[V_mod == 0] = 0.5

    export = Q*np.minimum(V_mod*eta, 1)

    left_exports = np.roll((V[...,0] < 0)*export*repartition, -1, axis=0)
    left_exports[-1,:] = 0
    right_exports = np.roll((V[...,0] > 0)*export*repartition, 1, axis=0)
    right_exports[0,:] = 0
    down_exports = np.roll((V[...,1] < 0)*export*(1 - repartition), -1, axis=1)
    down_exports[:,-1] = 0
    up_exports = np.roll((V[...,1] > 0)*export*(1 - repartition), 1, axis=1)
    up_exports[:,0] = 0

    dQ = -export + left_exports + right_exports + down_exports + up_exports

    Q += dQ

    Qs.append(np.copy(Q))

    if np.abs(dQ).max() < 1e-5:
        break




In [ ]:
np.max(Qs[-1])

In [ ]:
folder = 'erosion-movie-1'
if not os.path.exists(folder):
    os.makedirs(folder)
else:
    raise FileExistsError()

frame = 1
for i in tqdm(range(len(Qs))):

    if i % 1 == 0:
        plt.close(1)
        fig, ax = plt.subplots(num=1, figsize=(9,6))

        # plt.imshow(height_map0, cmap='gray')
        plt.pcolormesh(X,Y,Qs[i], cmap='Blues', vmin=0, vmax=1.5)
        plt.axis('equal')
        plt.colorbar()

        fig.tight_layout()

        fig.savefig(f'{folder}/{frame:04d}.png', dpi=200)
        frame += 1

In [ ]:
plt.close('all')

## Accounting for water height

The previous simulation has water accumulating in the lowest pixels. But that is unrealistic, as we want rather the formation of lakes, where the water fills the lowest basins in the height map.
However, we need to be careful when we modify the height map to avoid the formation of waves.

### Test anti-slosh

In [ ]:
# test water pour
height_map0 = np.zeros_like(height_map0)
Q = np.zeros_like(height_map0)
Q[Q.shape[0]//2, Q.shape[1]//2] = 100
Q[Q.shape[0]//2+1, Q.shape[1]//2] = 100
Q[Q.shape[0]//2, Q.shape[1]//2+1] = 100
Q[Q.shape[0]//2+1, Q.shape[1]//2+1] = 100

In [ ]:
Q = np.ones_like(height_map0)
water_height = 0.1
eta = 10

anti_slosh = 3


H = np.copy(height_map0) + water_height*Q
# Q = np.zeros_like(height_map0)
V = np.stack(np.gradient(-H), axis=-1)

def roll(array, direction='right'):
    if direction == 'left':
        a = np.roll(array, -1, axis=0)
        a[-1,:] = 0
    elif direction == 'right':
        a = np.roll(array, 1, axis=0)
        a[0,:] = 0
    elif direction == 'down':
        a = np.roll(array, -1, axis=1)
        a[:,-1] = 0
    elif direction == 'up':
        a = np.roll(array, 1, axis=1)
        a[:,0] = 0
    else:
        raise ValueError()

    return a

Qs = [np.copy(Q)]
# iterate
for i in tqdm(range(3000)):
    V = np.stack(np.gradient(-H), axis=-1)
    V_mod = np.abs(V[...,0]) + np.abs(V[...,1])
    repartition = np.abs(V[...,0])/V_mod # fraction of flow in x direction at each point
    repartition[V_mod == 0] = 0.5

    max_export = Q*np.minimum(V_mod*eta, 1)

    max_left_export = np.maximum((H - roll(H, 'right'))/water_height/(anti_slosh + 1), 0)
    max_right_export = np.maximum((H - roll(H, 'left'))/water_height/(anti_slosh + 1), 0)
    max_down_export = np.maximum((H - roll(H, 'up'))/water_height/(anti_slosh + 1), 0)
    max_up_export = np.maximum((H - roll(H, 'down'))/water_height/(anti_slosh + 1), 0)

    left_exports = np.minimum((V[...,0] < 0)*max_export*repartition, max_left_export)
    right_exports = np.minimum((V[...,0] > 0)*max_export*repartition, max_right_export)
    down_exports = np.minimum((V[...,1] < 0)*max_export*(1 - repartition), max_down_export)
    up_exports = np.minimum((V[...,1] > 0)*max_export*(1 - repartition), max_up_export)

    export = left_exports + right_exports + down_exports + up_exports

    left_exports = roll(left_exports, 'left')
    right_exports = roll(right_exports, 'right')
    down_exports = roll(down_exports, 'down')
    up_exports = roll(up_exports, 'up')

    dQ = -export + left_exports + right_exports + down_exports + up_exports

    Q += dQ
    H += water_height*dQ

    Qs.append(np.copy(Q))

    

In [ ]:
folder = 'erosion-movie-4'
if not os.path.exists(folder):
    os.makedirs(folder)
else:
    raise FileExistsError()

frame = 1
for i in tqdm(range(len(Qs))):

    if i % 1 == 0:
        plt.close(2)
        fig, ax = plt.subplots(num=2, figsize=(9,6))

        # plt.imshow(height_map0, cmap='gray')
        plt.pcolormesh(X,Y,Qs[i], cmap='Blues', vmin=0, vmax=1.5)
        plt.axis('equal')
        plt.colorbar()

        fig.tight_layout()

        fig.savefig(f'{folder}/{frame:04d}.png', dpi=200)
        frame += 1

In [ ]:
folder = 'erosion-movie-height-4'
if not os.path.exists(folder):
    os.makedirs(folder)
else:
    raise FileExistsError()

frame = 1
for i in tqdm(range(len(Qs))):

    if i % 30 == 0:
        plt.close(2)
        fig, ax = plt.subplots(num=2, figsize=(9,6))

        # plt.imshow(height_map0, cmap='gray')
        plt.pcolormesh(X,Y,height_map0 + water_height*Qs[i], cmap='gray',
                        # vmin=0, vmax=1.5
                        )
        plt.axis('equal')
        plt.colorbar()

        fig.tight_layout()

        fig.savefig(f'{folder}/{frame:04d}.png', dpi=200)
        frame += 1

### Trying a different approach

Anti slosh can still generate waves because multiple neighbors can come to fill the same grid point, allowing it to become higher than its neighbors and initiating oscillations.
Also, using the gradient will get us stuck in symmetric situations.

We can simplify computing the height differences between neighboring pixels and then the flow. Also, it is quite easy to add precipitation and evaporation

In [ ]:
# test water pour
height_map0 = np.zeros_like(height_map0)
Q = np.zeros_like(height_map0)
Q[Q.shape[0]//2, Q.shape[1]//2] = 100
Q[Q.shape[0]//2+1, Q.shape[1]//2] = 100
Q[Q.shape[0]//2, Q.shape[1]//2+1] = 100
Q[Q.shape[0]//2+1, Q.shape[1]//2+1] = 100

In [ ]:
Q = np.ones_like(height_map0)
water_height = 0.1
eta = 0.1
eps_tol = 1e-5

P_freq = 87
evaporation = 0.02


H = np.copy(height_map0) + water_height*Q
# Q = np.zeros_like(height_map0)
V = np.stack(np.gradient(-H), axis=-1)

directions = ['left', 'down', 'right', 'up']
def roll(array, direction=0):
    if direction in [0, 'left']:
        a = np.roll(array, -1, axis=0)
        a[-1,:] = 0
    elif direction in [1, 'down']:
        a = np.roll(array, -1, axis=1)
        a[:,-1] = 0
    elif direction in [2, 'right']:
        a = np.roll(array, 1, axis=0)
        a[0,:] = 0
    elif direction in [3, 'up']:
        a = np.roll(array, 1, axis=1)
        a[:,0] = 0
    else:
        raise ValueError()

    return a

Qs = [np.copy(Q)]
# iterate
for i in tqdm(range(30000)):

    # the exports to the left are computed comparing to the left neighbor, which requires rolling the height map right
    exports = eta*np.stack([np.maximum((H - roll(H, ((d+2) % 4)))/water_height, 0) for d in range(4)], axis=-1) 

    total_export = np.sum(exports, axis=-1)
    # adjust export to be less than Q
    total_export_ = np.minimum(total_export, Q)
    total_export[total_export == 0] = 1
    exports = (exports.T * total_export_.T / total_export.T).T
    total_export = total_export_

    # compute imports by properly rolling exports
    imports = np.stack([roll(exports[...,d], d) for d in range(4)], axis=-1)

    total_import = np.sum(imports, axis=-1)

    E = np.ones_like(Q) * (Q > 0) * evaporation
    P = np.zeros_like(Q)
    if i % P_freq == 0:
        P += 1


    dQ = total_import - total_export + P - E

    # clip so that Q stays above 0
    dQ = np.maximum(Q + dQ, 0) - Q

    if np.max(np.abs(dQ)) < eps_tol:
        break

    Q += dQ
    H += water_height*dQ

    Qs.append(np.copy(Q))

    

In [ ]:
np.sum(Qs[100])

In [ ]:
folder = 'erosion-movie-6'
if not os.path.exists(folder):
    os.makedirs(folder)
else:
    raise FileExistsError()

frame = 1
for i in tqdm(range(len(Qs))):

    if i % 100 == 0:
        plt.close(2)
        fig, ax = plt.subplots(num=2, figsize=(9,6))

        # plt.imshow(height_map0, cmap='gray')
        plt.pcolormesh(X,Y,Qs[i], cmap='Blues', vmin=0, vmax=1.5)
        plt.axis('equal')
        plt.colorbar()

        fig.tight_layout()

        fig.savefig(f'{folder}/{frame:04d}.png', dpi=200)
        frame += 1

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 

plt.close(1)
fig = plt.figure(num=1, figsize=(9,6))
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(X, Y, height_map0 - 0.01, color='gray')
ax.plot_surface(X, Y, height_map0 + water_height*Qs[-1], color='blue', alpha=0.5)


fig.tight_layout()

In [ ]:
plt.close(1)
fig, ax = plt.subplots(num=1, figsize=(9,6))

plt.plot([np.sum(Q_) for Q_ in Qs], label='Q')

fig.tight_layout()

This seems to work pretty well, though we might have problems with inertia. But maybe it is good to ditch the idea of inertia... we'll see